$$ p_k = -H_k \cdot \nabla f(x_k) \text{ --- направление на k-м шаге} $$
$$ \alpha_k = argmin\ \alpha > 0: f(x_k + \alpha p_k \text{ --- размер шага} $$
$$ x_{k + 1} = x_k + \alpha_k p_k \text{ --- новая точка} $$
$$ y_k = \nabla f(x_{k + 1}) - \nabla f(x_k) $$
$$ s_k = x_{k + 1} - x_k $$
$$ \rho_k = \frac{1}{y_k^T \cdot s_k $$
$$ H_{k + 1} = (I - \rho_k \cdot s_k \cdot y_k^T) \cdot H \cdot (I - \rho_k \cdot y_k \cdot s_k^T) + \rho \cdot s_k \cdot s_k^T \text{ --- обновляем Гессиан} $$

In [ ]:
from matplotlib import pyplot as plt
import math
import numpy as np
from scipy.optimize import line_search
from dataset_generator import Generator


# TODO: - Fix print_full_grad
#       - Uniform bfgs
#       - Make our own Wolfe based line search


class file_info_3d:
    def __init__(self, X=None, Y=None, f=None, x0=None):
        self.X = X
        self.Y = Y
        self.Z = np.vectorize(lambda x, y: f(np.array([x, y])))(X, Y)
        self.f = f
        self.x0 = x0


def print_full_grad(file_info, list_result, list_label, title='Градиентный спуск на графике функции', elev = 30, azim = 80, filename='', filename_extension='.png', dpi=1024, isshow=True):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(projection='3d')

    x = list_result[:, 0]
    y = list_result[:, 1]
    z = np.vectorize(lambda x, y: file_info.f(np.array([x, y])))(x, y)
    ax.plot(x, y, marker='.', markersize=10, markerfacecolor='black', zs=z, label=list_label, linewidth = 2)
    print(f'{list_label:15} ==> {file_info.f(list_result[-1]):10f} in [{list_result[-1][0]:10f}, {list_result[-1][1]:10f}]')

    ax.plot_surface(file_info.X, file_info.Y, file_info.Z, linewidth=0, antialiased=False, shade=False, cmap='terrain')
    ax.view_init(elev=elev, azim=azim)

    # Установка отступа между графиком и значениями осей
    ax.tick_params(pad=10)

    # Добавление легенды
    if len(list_label) > 0:
        ax.legend(loc='upper left')

    # Установка размера шрифта для подписей осей
    ax.tick_params(axis='x', labelsize=10)
    ax.tick_params(axis='y', labelsize=10)
    ax.tick_params(axis='z', labelsize=10)

    # Добавление заголовка и подписей осей
    if title != '':
        plt.title(title)

    ax.set_xlabel('Ось X', labelpad=20.0)
    ax.set_ylabel('Ось Y', labelpad=20.0)
    ax.set_zlabel('Ось f(x, y)', labelpad=20.0)

    if(filename != ''):
        plt.savefig(filename + filename_extension, dpi=dpi, bbox_inches=0, transparent=True)

    plt.show()


def print_lines_grad(file_info_3d, list_result, list_label, title='Градиентный спуск на уровнях функции', filename='',
                     filename_extension='.png', dpi=1024):
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111)

    levels = np.unique(sorted([file_info_3d.f(p) for p in list_result]))
    ax.contour(file_info_3d.X, file_info_3d.Y, file_info_3d.Z, levels=levels, colors='red', antialiased=True,
               linewidths=1.0)

    x = list_result[:, 0]
    y = list_result[:, 1]
    ax.plot(x, y, marker='.', markersize=10, markerfacecolor='black', label=list_label, linewidth=2)
    print(
        f'{list_label:15} ==> '
        f'{file_info_3d.f(list_result[-1]):10f} in [{list_result[-1][0]:10f}, {list_result[-1][1]:10f}]'
    )

    # Добавление заголовка и подписей осей
    if title != '':
        plt.title(title)

    # Добавление легенды
    if len(list_label) > 0:
        plt.legend(loc='upper left')

    if filename != '':
        plt.savefig(filename + '_lines' + filename_extension, dpi=dpi, bbox_inches=0, transparent=True)

    plt.show()


def grad(x, delta=1e-9):
    # """
    # Функция вычисления градиента в заданной точке с константной точностью
    #
    # Аргументы:
    # f -- функция
    # x -- точка
    #
    # Возвращает:
    # ans -- градиент функции в точке x
    # """
    #
    # n = len(x)
    # xd = np.copy(x)
    # ans = np.zeros(n)
    #
    # for i in range(n):
    #     xd[i] += delta
    #     ans[i] = np.divide(f(xd) - f(x), delta)
    #     xd[i] -= delta
    #
    # return ans

    return np.array([2 * (200 * x[0] ** 3 - 200 * x[0] * x[1] + x[0] - 1), 200 * (x[1] - x[0] ** 2)])


def f(x):
    return (1 - x[0]) ** 2 + 100 * (x[1] - x[0] ** 2) ** 2


def wolfe_line_search(f, x_init, d, alpha=100.0, c1=1e-4, c2=0.9, max_iter=150, eps=1e-9):
    def less_or_equal(lhs, rhs):
        return lhs < rhs or math.isclose(lhs, rhs)

    def armijo(alpha):
        left = f(x_init + alpha * d)
        right = f(x_init) + c1 * alpha * np.dot(grad(f, x_init), d)

        return less_or_equal(left, right)

    def curvature(alpha):
        left = np.dot(grad(f, x_init + alpha * d), d)
        right = c2 * np.dot(grad(f, x_init), d)

        return less_or_equal(right, left)

    iter_count = 0
    alpha_low = 0.
    alpha_high = np.inf
    alpha_prev = 0
    alpha_cur = alpha

    while iter_count < max_iter:
        if not armijo(alpha_cur):
            alpha_high = alpha_cur
            alpha_cur = (alpha_low + alpha_high) * 0.5
        elif not curvature(alpha_cur):
            alpha_low = alpha_cur
            if np.isinf(alpha_high):
                alpha_cur = 2 * alpha_low
            else:
                alpha_cur = (alpha_low + alpha_high) * 0.5
        else:
            return alpha_cur

        if np.abs(alpha_cur - alpha_prev) < 1e-6:
            break

        alpha_prev = alpha_cur
        iter_count += 1

    return alpha_cur


def bfgs(initial_w, func, grad_func, max_iter=1000, tolerance=1e-9, eps=1e-10):
    result = [np.copy(initial_w)]
    current_x = np.copy(initial_w)
    n = len(initial_w)
    H = np.identity(n)

    for i in range(max_iter):
        grad_f = grad_func(current_x)
        p = -1 * H @ grad_f
        # alpha = wolfe_line_search(func, current_x, p)
        alpha, _, _, _, _, _ = line_search(func, grad_func, current_x, p)

        if alpha is None:
            alpha = 0

        # print(f'Wolfe alpha: {alpha}')

        new_x = current_x + alpha * p
        new_grad_f = grad_func(new_x)

        y = new_grad_f - grad_f
        s = new_x - current_x

        rho = 1 / (y.T @ s + eps)
        H = (np.identity(n) - rho * y @ s.T) @ H @ (np.identity(n) - rho * y @ s.T) + rho * s @ s.T

        if np.linalg.norm(new_x - current_x, 2) < tolerance:
            print("too small")
            break

        current_x = new_x
        result.append(current_x)

    return result


# Generate synthetic data.
# PARAMETERS
density = 8000
dots_count = 500
variance = 0.1
dist = 2.5
radius = 0.8
initial_x = np.array([-2, 0])
# ===========

# gen = Generator(func_without_f)
# X, Y, Dataset_X, Dataset_Y = gen.generate(dots_count, dist, density, variance, weights, absolute=False)

# Apply Gauss-Newton method.
max_iter = 10000
result = bfgs(initial_x, f, grad, max_iter=max_iter)

print(f'Result: {result[-1]}\nFunction value: {f(result[-1])}')

# Plot style:
plt.style.use('default')
_ = plt.figure(figsize=(8, 8))
# ===========

# print(f"Result: {result[-1]}")
# print(f'Loss value: {loss(result[-1])}')
#
# plt.axis('equal')
# plt.scatter(Dataset_X, Dataset_Y, label='Data', color='gray', alpha=0.5, s=20.8, antialiased=True)
# plt.plot(X, Y, label='Real', color='lime', antialiased=True, linewidth=3)
# plt.plot(X, func_without_f(X, result[-1]), label='Fit', color='coral', antialiased=True, linewidth=1.7)
# plt.legend()
# plt.show()

x_lin = np.linspace(-5, 5, 100, dtype=float)
y_lin = np.linspace(-2, 8, 100, dtype=float)
X, Y = np.meshgrid(x_lin, y_lin)

f_info = file_info_3d(X, Y, f, initial_x)
print_lines_grad(f_info, np.array(result), 'BFGS')

print_full_grad(f_info, np.array(result), 'BFGS')